# Limpieza del Data

In [69]:
import pandas as pd
import string
import re 

In [70]:
#Cargamos los datas
data1= pd.read_csv('phishing_site_urls.csv') # kaggle
data2= pd.read_csv('URL dataset.csv') #data phishing

In [71]:
# Obtenemos informacion rapida de los datas
data1.info()
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549346 entries, 0 to 549345
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     549346 non-null  object
 1   Label   549346 non-null  object
dtypes: object(2)
memory usage: 8.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450176 entries, 0 to 450175
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     450176 non-null  object
 1   type    450176 non-null  object
dtypes: object(2)
memory usage: 6.9+ MB


In [72]:
# Verificar valores nulos en cada columna 
valores_nulos_data1 = data1.isnull().sum() 
valores_nulos_data2 = data2.isnull().sum() 
print("Valores nulos en data1:\n", valores_nulos_data1) 
print("\nValores nulos en data2:\n", valores_nulos_data2) 


Valores nulos en data1:
 URL      0
Label    0
dtype: int64

Valores nulos en data2:
 url     0
type    0
dtype: int64


In [73]:
# Verificar espacios vacíos en cada columna 
espacios_vacios_data1 = data1.apply(lambda x: isinstance(x, str) and x.strip() == '').sum() 
espacios_vacios_data2 = data2.apply(lambda x: isinstance(x, str) and x.strip() == '').sum()
print("\nEspacios vacíos en data1:\n", espacios_vacios_data1) 
print("\nEspacios vacíos en data2:\n", espacios_vacios_data2)


Espacios vacíos en data1:
 0

Espacios vacíos en data2:
 0


In [74]:
# Exploramos los valores unicos de las columnas 'Label' y 'type' 
valores = data1['Label'].unique()
print(valores)

valores = data2['type'].unique()
print(valores)

['bad' 'good']
['legitimate' 'phishing']


In [75]:
# Eliminamos filas duplicadas 
for df in [data1, data2]:
    duplicados= df.duplicated().sum()
    print(f"Hay {duplicados} filas duplicadas")
    df.drop_duplicates(inplace=True)
    eliminado=duplicados
    print(f"Se ha eliminado {eliminado} filas")
    
    print(f"Datas sin filas duplicadas")


Hay 42150 filas duplicadas
Se ha eliminado 42150 filas
Datas sin filas duplicadas
Hay 0 filas duplicadas
Se ha eliminado 0 filas
Datas sin filas duplicadas


# Concatenamos datas

In [76]:
#renombrar columnas 
data1.rename(columns={'URL': 'url', 'Label': 'status'}, inplace=True)
data2.rename(columns={'type': 'status'}, inplace=True)


#unir datas
datas=pd.concat([data1,data2], ignore_index=True)

#actualizamos las etiquetas del data1
datas['status']=datas['status'].replace({'bad': '-1', 'good': '1', 'phishing': '-1', 'legitimate':'1'})
print("Las datas han sido unidas")
print(datas.columns)

Las datas han sido unidas
Index(['url', 'status'], dtype='object')


In [77]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957372 entries, 0 to 957371
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     957372 non-null  object
 1   status  957372 non-null  object
dtypes: object(2)
memory usage: 14.6+ MB


In [78]:
# La columna status tiene como valores de 1=legitimate, -1=phishing
conteo_datas= datas['status'].value_counts()
print(conteo_datas)


status
1     738635
-1    218737
Name: count, dtype: int64


In [79]:
#exploramos los prefijos existentes en la columna url
# Función para extraer el prefijo
def extract_prefix(url):
    # Asegurarnos de trabajar con strings
    url = str(url)
    prefixes = ['https://', 'http://', 'ftp://', 'ftps://', 'mailto://', 'file://','data://', 'tel://', 'ldap://','news://', 'nntp://','telnet://', 'gopher://']
    for prefix in prefixes:
        if url.startswith(prefix):
            return prefix[:-3]  # Devolver 'https', 'http', o 'ftp' etc.....
    return '0'

# Aplicar la función para extraer prefijos
datas['prefix'] = datas['url'].apply(extract_prefix)

# Contar la cantidad de cada prefijo
prefix_counts = datas['prefix'].value_counts()

print("Contador de prefijos en las URLs:")
print(prefix_counts)


Contador de prefijos en las URLs:
prefix
0        507119
https    352192
http      98048
ftp          13
Name: count, dtype: int64


In [80]:
#Subdominios y host (IP)
# Contar URLs que comienzan con www.
www_count = datas['url'].str.startswith('www.').sum()

# Contar URLs que son direcciones IP
ip_count = datas['url'].str.match(r'^\d{1,3}(\.\d{1,3}){3}').sum()

# Mostrar los resultados
print(f"Cantidad de URLs que comienzan con 'www.': {www_count}")
print(f"Cantidad de URLs que corresponden a direcciones IP: {ip_count}")


Cantidad de URLs que comienzan con 'www.': 45202
Cantidad de URLs que corresponden a direcciones IP: 3703


In [81]:
# Se evidenció urls con error en el prefijo, status=legitimate
# Filtrar URLs que comiencen con prefijos mal formados
urls_malformadas = datas[datas['url'].str.startswith(('httpss://', 'htps://'))]

# Mostrar las URLs malformadas
print("URLs con prefijos incorrectos:")
print(urls_malformadas)


URLs con prefijos incorrectos:
                                                      url status prefix
849512    httpss://sites.google.com/site/lettheshowbegin/      1      0
849638           httpss://www.googlemerchandisestore.com/      1      0
849643                        httpss://www.microsoft.com/      1      0
849865                        httpss://www.bluesteps.com/      1      0
850146                  httpss://news.google.com/?topic=b      1      0
850177                httpss://www.thrivent.com/magazine/      1      0
850278                           httpss://fancybeads.com/      1      0
850313                             httpss://sentiell.com/      1      0
850321                httpss://www.tektiktattoosupply.be/      1      0
850527  httpss://people.math.osu.edu/events/connes/Con...      1      0
850783  httpss://mail.cms.math.ca/mailman/listinfo/web...      1      0
850904            httpss://www.floralcraftsupplies.co.uk/      1      0
850955                      https

In [82]:
datas.columns


Index(['url', 'status', 'prefix'], dtype='object')

In [83]:
datas

,url,status,prefix
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,-1,0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,-1,0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,-1,0
3,mail.printakid.com/www.online.americanexpress....,-1,0
4,thewhiskeydregs.com/wp-content/themes/widescre...,-1,0
...,...,...,...
957367,http://ecct-it.com/docmmmnn/aptgd/index.php,-1,http
957368,http://faboleena.com/js/infortis/jquery/plugin...,-1,http
957369,http://faboleena.com/js/infortis/jquery/plugin...,-1,http
957370,http://atualizapj.com/,-1,http


In [84]:

# Se añade columnas de prefijos 
# Funcion para verificar si la url usa ciertos prefijos
def has_prefix(url, prefix): 
    return 1 if url.startswith(prefix) else 0 
def starts_with_ip(url):
    ip_pattern = re.compile(r'^(\d{1,3}\.){3}\d{1,3}')
    return 1 if ip_pattern.match(url) else 0

# Crear columnas para cada prefijo en el DataFrame 
datas['has_https'] = datas['url'].apply(lambda x: has_prefix(x, 'https://')) 
datas['has_http'] = datas['url'].apply(lambda x: has_prefix(x, 'http://')) 
datas['has_ftp'] = datas['url'].apply(lambda x: has_prefix(x, 'ftp://')) 
datas['has_www'] = datas['url'].apply(lambda x: has_prefix(x, 'www.'))
datas['has_ip']= datas['url'].apply(starts_with_ip)

print(datas)

                                                      url status prefix  \
0       nobell.it/70ffb52d079109dca5664cce6f317373782/...     -1      0   
1       www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...     -1      0   
2       serviciosbys.com/paypal.cgi.bin.get-into.herf....     -1      0   
3       mail.printakid.com/www.online.americanexpress....     -1      0   
4       thewhiskeydregs.com/wp-content/themes/widescre...     -1      0   
...                                                   ...    ...    ...   
957367        http://ecct-it.com/docmmmnn/aptgd/index.php     -1   http   
957368  http://faboleena.com/js/infortis/jquery/plugin...     -1   http   
957369  http://faboleena.com/js/infortis/jquery/plugin...     -1   http   
957370                             http://atualizapj.com/     -1   http   
957371  http://writeassociate.com/test/Portal/inicio/I...     -1   http   

        has_https  has_http  has_ftp  has_www  has_ip  
0               0         0        0       

In [85]:
#Funciones 
#Funcion para calcular el largo del host

def host_length(url):
    parts = url.split('/')
    if len(parts) > 2:
        return len(parts[2])
    else:
        return -1  # O cualquier otro valor que quieras usar para URLs no válidas

def longest_word_length(url):
    words = url.split('/')
    return max((len(word) for word in words), default=-1)

def shortest_word_length(url):
    words = url.split('/')
    return min((len(word) for word in words if word), default=-1)

def average_word_length(url):
    words = url.split('/')
    return sum(len(word) for word in words) / len(words) if words else -1

def digit_ratio(url):
    if len(url) == 0:
        return -1
    digits = sum(c.isdigit() for c in url)
    return digits / len(url)

def host_digit_ratio(url):
    parts = url.split('/')
    if len(parts) > 2:
        host = parts[2]
        digits = sum(c.isdigit() for c in host)
        return digits / len(host) if len(host) > 0 else 0
    else:
        return -1

# Añadir nuevas columnas antes de estandarizar las URLs 
datas['url_length'] =datas['url'].apply(len)
datas['largo_host'] = datas['url'].apply(host_length) 
datas['longitud_palabra_mas_larga'] = datas['url'].apply(longest_word_length) 
datas['longitud_palabra_mas_corta'] = datas['url'].apply(shortest_word_length) 
datas['promedio_palabra_por_fila'] = datas['url'].apply(average_word_length) 
datas['ratio_digitos_url'] = datas['url'].apply(digit_ratio) 
datas['ratio_digitos_host'] = datas['url'].apply(host_digit_ratio)



In [86]:


# Función para eliminar espacios en blanco
def clean_url(url):
    return re.sub(r'\s+', '', str(url))

# Aplicar la función para limpiar las URLs sin eliminar prefijos
datas['url'] = datas['url'].apply(clean_url)

# Eliminar filas duplicadas
duplicados = datas.duplicated().sum()
print(f"Hay {duplicados} filas duplicadas")
datas.drop_duplicates(inplace=True)
print(f"Se han eliminado {duplicados} filas")
print("Datas sin filas duplicadas")

# Mostrar el DataFrame limpio
print(datas.head())


Hay 39 filas duplicadas
Se han eliminado 39 filas
Datas sin filas duplicadas
                                                 url status prefix  has_https  \
0  nobell.it/70ffb52d079109dca5664cce6f317373782/...     -1      0          0   
1  www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...     -1      0          0   
2  serviciosbys.com/paypal.cgi.bin.get-into.herf....     -1      0          0   
3  mail.printakid.com/www.online.americanexpress....     -1      0          0   
4  thewhiskeydregs.com/wp-content/themes/widescre...     -1      0          0   

   has_http  has_ftp  has_www  has_ip  url_length  largo_host  \
0         0        0        0       0         225          15   
1         0        0        1       0          81           9   
2         0        0        0       0         177           4   
3         0        0        0       0          60          10   
4         0        0        0       0         116           6   

   longitud_palabra_mas_larga  longitud_palab

In [87]:

# Definir el alfabeto y otros caracteres
alfabeto_minuscula = string.ascii_lowercase
alfabeto_mayuscula = string.ascii_uppercase
caracteres_str = "!@#$%^&*()_+-=[]{}|;:',.<>?/\\"
caracteres = list(caracteres_str)
numeros = "0123456789"
# Definir los prefijos de interés 
pref_interes = ['https', 'http', 'ftp', 'ftps', 'mailto', 'file', 'data', 'tel', 'ldap', 'news', 'nntp', 'telnet', 'gopher', 'www', 'm', 'api', 'rss', 'atom']

# Funciones para características de las URLs
def caracteres_repet(url):
    count = 0
    for i in range(1, len(url)):
        if url[i] == url[i-1]:
            count += 1
    return count

def count_spaces(url):
    return url.count(' ')

def count_chars(url, chars):
    return {char: url.count(char) for char in chars}

def count_subdomains(url):
    url = re.sub(r'^(http://|https://|ftp://|www\.)', '', url)
    return url.count('.')




# Función para identificar prefijos fuera de lugar 
def find_misplaced_prefix(url):
    # Verificar si el prefijo aparece fuera del inicio de la URL 
    for prefijo in pref_interes:
        pattern = fr'^[^{prefijo}].*{re.escape(prefijo)}'
        if re.search(pattern, url) and not url.startswith(f"{prefijo}.")and not url.startswith(f"{prefijo}:"):
            return 1 #prefijo fuera de lugar
    return 0 #todos los prefijos estan en su lugar correcto


# Calcular nuevas características
df_new = pd.DataFrame({
    'repeated_chars': datas['url'].apply(caracteres_repet),
    'space_count': datas['url'].apply(count_spaces),
    'num_special_chars': datas['url'].apply(lambda x: sum(x.count(c) for c in caracteres)),
    'num_subdomains': datas['url'].apply(lambda x: x.count('.')),
    'prefijos_fuera_de_lugar': datas['url'].apply(find_misplaced_prefix)
})

# Añadir columnas para cada letra minúscula
for char in alfabeto_minuscula:
    df_new[f'count_{char}_lower'] = datas['url'].apply(lambda x: x.count(char))

# Añadir columnas para cada letra MAYUSCULA
for char in alfabeto_mayuscula:
    df_new[f'count_{char}_upper'] = datas['url'].apply(lambda x: x.count(char))

# Añadir columnas para cada número
for num in numeros:
    df_new[num] = datas['url'].apply(lambda x: x.count(num))

# Añadir columnas para cada carácter especial
for char in caracteres:
    col_name = f"count_{char}"
    df_new[col_name] = datas['url'].apply(lambda x: x.count(char))

# 

# Unir las nuevas columnas al DataFrame original
datas = pd.concat([datas, df_new], axis=1)

# Mover la columna 'status' al final de las columnas
columns = [col for col in datas.columns if col != 'status'] + ['status']
datas = datas[columns]

# Mostrar el DataFrame con las nuevas columnas
print(datas.head())


                                                 url prefix  has_https  \
0  nobell.it/70ffb52d079109dca5664cce6f317373782/...      0          0   
1  www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...      0          0   
2  serviciosbys.com/paypal.cgi.bin.get-into.herf....      0          0   
3  mail.printakid.com/www.online.americanexpress....      0          0   
4  thewhiskeydregs.com/wp-content/themes/widescre...      0          0   

   has_http  has_ftp  has_www  has_ip  url_length  largo_host  \
0         0        0        0       0         225          15   
1         0        0        1       0          81           9   
2         0        0        0       0         177           4   
3         0        0        0       0          60          10   
4         0        0        0       0         116           6   

   longitud_palabra_mas_larga  ...  count_:  count_'  count_,  count_.  \
0                          48  ...        0        0        0        6   
1               

In [88]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 957333 entries, 0 to 957371
Columns: 111 entries, url to status
dtypes: float64(3), int64(105), object(3)
memory usage: 818.0+ MB


In [89]:
datas

,url,prefix,has_https,has_http,has_ftp,has_www,has_ip,url_length,largo_host,longitud_palabra_mas_larga,...,count_:,count_',"count_,",count_.,count_<,count_>,count_?,count_/,count_\,status
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,0,0,0,0,0,0,225,15,48,...,0,0,0,6,0,0,1,10,0,-1
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,0,0,0,0,1,0,81,9,30,...,0,0,0,5,0,0,0,4,0,-1
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,0,0,0,0,0,0,177,4,76,...,0,0,0,7,0,0,0,11,0,-1
3,mail.printakid.com/www.online.americanexpress....,0,0,0,0,0,0,60,10,30,...,0,0,0,6,0,0,0,2,0,-1
4,thewhiskeydregs.com/wp-content/themes/widescre...,0,0,0,0,0,0,116,6,24,...,0,0,0,1,0,0,1,10,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957367,http://ecct-it.com/docmmmnn/aptgd/index.php,http,0,1,0,0,0,43,11,11,...,1,0,0,2,0,0,0,5,0,-1
957368,http://faboleena.com/js/infortis/jquery/plugin...,http,0,1,0,0,0,159,13,32,...,1,0,0,2,0,0,0,14,0,-1
957369,http://faboleena.com/js/infortis/jquery/plugin...,http,0,1,0,0,0,147,13,32,...,1,0,0,1,0,0,0,14,0,-1
957370,http://atualizapj.com/,http,0,1,0,0,0,22,14,14,...,1,0,0,1,0,0,0,3,0,-1


In [90]:

# Función para limpiar caracteres ilegales y espacios en blanco
def clean_spaces(url):
    return re.sub(r'\s+', '', url)

# Aplicar la limpieza de caracteres ilegales y espacios en blanco a la columna 'url'
datas['url'] = datas['url'].apply(clean_spaces)

# Verificar y encontrar duplicados en el DataFrame
duplicados = datas[datas.duplicated(subset=['url'], keep=False)]

# Contar y mostrar duplicados
total_duplicados = duplicados.shape[0]
print(f"Total de filas duplicadas: {total_duplicados}")
print(duplicados)

# Eliminar duplicados manteniendo solo la primera ocurrencia
datas_sin_duplicados = datas.drop_duplicates(subset=['url'])
datas=datas_sin_duplicados





Total de filas duplicadas: 28
                                                      url prefix  has_https  \
34026   'update-information010.bjit.com.au/?/cgi-bin/w...      0          0   
34164   'update-information010.bjit.com.au/?/cgi-bin/w...      0          0   
98957   149.56.146.132/~wellsfargo0o/cgi-bin/wellsfarg...      0          0   
98962   149.56.146.132/~wellsfargo0o/cgi-bin/wellsfarg...      0          0   
108281  ankarasoveyapi.com/Cesc/bookmark/ii.php?rand=1...      0          0   
109133  ankarasoveyapi.com/Cesc/bookmark/ii.php?rand=1...      0          0   
242602                                tommyhumphreys.com/      0          0   
482296               pastebin.com/download.php?i=1YzPHtum      0          0   
482487               pastebin.com/download.php?i=1YzPHtum      0          0   
488447  intent.nofrillspace.com/users/web11_focus/3807...      0          0   
488448  mister.nofrillspace.com/users/web8_dice/3791/s...      0          0   
488449  intent.nofrill

In [91]:
import re
#filtramos las urls con direccion ip
ip_pattern = re.compile(r'^(\d{1,3}\.){3}\d{1,3}')
filtered = datas[datas['url'].str.match(ip_pattern)]

# Contar el número total de duplicados 
duplicados = filtered[filtered.duplicated(subset='url', keep=False)]
conteo_duplicados = duplicados['url'].value_counts()
total_duplicados = conteo_duplicados.sum() 
print(f"Total de URLs duplicadas: {total_duplicados}")

#eliminado duplicados del data
datas_actualizado= datas.drop_duplicates(subset='url', keep='first')
print("Se ha eliminado URLs duplicadas por IP")

# actualizar el data original sin las urls duplicadas
datas_actualizado= datas[~datas.index.isin(duplicados.index)]
#verificamos q se han eliminado
ip_pattern2 = re.compile(r'^(\d{1,3}\.){3}\d{1,3}')
filtered2 = datas_actualizado[datas_actualizado['url'].str.match(ip_pattern)]
conteo_duplicados2= filtered2.duplicated(subset='url', keep=False).sum()
print("verificando se tiene filas repetidas filtrado por ip")
print(conteo_duplicados2)


Total de URLs duplicadas: 0
Se ha eliminado URLs duplicadas por IP
verificando se tiene filas repetidas filtrado por ip
0


In [92]:
# Eliminamos filas con datos corrompidos
datas_actualizado.drop(index=range(18231, 18328), inplace=True)


In [93]:
datas_actualizado.to_csv('datos1.csv', index=False)

In [94]:
datos= pd.read_csv('datos1.csv', low_memory=False)

# Archivo 1: Solo columnas 'url' y 'status'
archivo1 = datos[['url', 'status']]
archivo1.to_csv('datos_solo_url_status.csv', index=False)

# Archivo 2: URLs con 'status' -1 (phishing) con todas las columnas
archivo2 = datos[datos['status'] == -1]
archivo2.to_csv('archivo_phishing.csv', index=False)

# Archivo 3: URLs con 'status' 1 (Legit) con todas las columnas
archivo3 = datos[datos['status'] == 1]
archivo3.to_csv('archivo_legitimos.csv', index=False)

print("Archivos guardados exitosamente.")


Archivos guardados exitosamente.


In [95]:
datos= pd.read_csv('datos1.csv', low_memory=False)
